In [ ]:
import numpy as np
import pandas as pd
import re
import unicodedata
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
nlp = spacy.load('en_core_web_lg')
#spacy.load('en_vectors_web_lg', vocab=nlp.vocab)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/train-csv/train.csv")
test = pd.read_csv("/kaggle/input/test-scv/test.csv")
sample_submission = pd.read_csv("/kaggle/input/sample-sub/sample_submission.csv")

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train['hash_string'] = train['text'].apply(
    lambda x: [i.replace('#', ' ').replace('  ', ' ').strip() if i.startswith('#') else '' for i  in x.split(' ')])


In [ ]:
train['hash_string'] = train['hash_string'].apply(lambda x: ' '.join(x).strip())

In [ ]:
for i in train['hash_string'].unique():
    print(i)

In [ ]:
def replace_elem(text):
    text = text.replace('#', ' ').replace('  ', ' ')
    text_clean = ' '.join(['' if i.startswith(('http','@', '&amp')) else i for i in text.split()]).strip()
    return text_clean

In [ ]:
def strip_accents(text):
  
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [ ]:
def text_prepro(text):
  
  stop_words = set(stopwords.words('english'))
  new_stopwords = ["i'm", "im", "there's", "that's", "wasnt"]
  new_stopwords_list = stop_words.union(new_stopwords)
  text = replace_elem(text)
  #text = ' '.join([i[1] if (i[1] == 'NNP' and 
                            #i[0].lower() not in stop_words and 
                            #pos_tag(text.split()).index(i) != 0) else i[0] for i  in pos_tag(text.split())])
  text = text.lower()
  word_tokens = word_tokenize(text) 
  filtered_sentence = [w for w in word_tokens if not w in new_stopwords_list and len(w) > 1]
  lemmatizer=WordNetLemmatizer()
  lematized_words = [lemmatizer.lemmatize(word) for word in filtered_sentence]
  stemmer= PorterStemmer()
  stemmed_words = [stemmer.stem(word) for word in lematized_words if word.isalpha() and len(word) > 1]
  stemmed_string = ' '.join(stemmed_words)
  stemmed_string = strip_accents(stemmed_string)
  
  return stemmed_string

In [ ]:
train['text'] = train['text'].apply(replace_elem)

In [ ]:
train['text'] = train['text'].apply(lambda x: ' '.join([i for i in x.split() if i.isalpha()]))

In [ ]:
#train['keyword'].fillna('empty', inplace = True)
no_kw = train[['text']][train['keyword'].isna()]
kw = train[['text', 'keyword']][train['keyword'].notna()]
no_kw['text'] = no_kw['text'].apply(lambda x: ' '.join([i for i in x.split() if i.isalpha()]))
kw['text'] = kw['text'].apply(lambda x: ' '.join([i for i in x.split() if i.isalpha()]))

In [ ]:
l1 = []
l2 = []
for i in no_kw['text']:
    lista_score = []
    lista_comments = []
    for e in kw['text'].unique():
        sim_scores = nlp(i).similarity(nlp(e))
        if len(lista_score) == 0:
            lista_score.append(sim_scores)
            lista_comments.append(e)
            
        elif (len(lista_score)) >0 and (lista_score[0] < sim_scores):
            lista_score[0] = sim_scores
            lista_comments[0] = e
        else:
            print("isn't higher")
    l1.append(lista_score)
    l2.append(lista_comments)

In [ ]:
no_kw['aux_comment'] = l2
no_kw['aux_score'] = l1

In [ ]:
no_kw['aux_comment'] = no_kw['aux_comment'].apply(lambda x: x[0])
kw = kw.drop_duplicates(subset='text', keep="first").rename(columns= {'text': 'original_coment'})
df_mix = pd.merge(no_kw, kw,
                 how = 'left',
                 left_on = 'aux_comment',
                 right_on = 'original_coment').drop(['original_coment'], axis = 1)

df_mix.to_csv("./kew_words.csv")
#df_mix.to_csv("/kaggle/keys.csv")

In [ ]:
extra_key_words = pd.read_csv('./kew_words.csv')

In [ ]:
train['keyword'].value_counts().to_frame()

In [ ]:
extra_key_words.head()

In [ ]:
extr_dict = dict(zip(extra_key_words.text, extra_key_words.keyword))
extr_dict

In [ ]:
train['keyword'] = np.where(train.keyword.isna(), train.text, train.keyword)
train.replace({"keyword": extr_dict}, inplace = True)

In [ ]:
train['keyword'].unique()

In [ ]:
train['keyword'] = train['keyword'].apply(lambda x: x.replace('%20', ' ').strip())

In [ ]:
train['text'] = train['text'] + ' '+ train['keyword']

In [ ]:
print(train['text'].unique())

In [ ]:
train['text_modif'] = train['text'].apply(text_prepro)

In [ ]:
from itertools import chain #- to check most the frequency of the words
from collections import Counter

In [ ]:
word_freq = Counter(chain.from_iterable(list(train['text_modif'].str.split())))
most_common = word_freq.most_common()

In [ ]:
most_common

In [ ]:
word_freq_t1 = Counter(chain.from_iterable(list(train['text_modif'][train['target'] == 1].str.split())))
most_common_t1 = word_freq_t1.most_common()

word_freq_t0 = Counter(chain.from_iterable(list(train['text_modif'][train['target'] == 0].str.split())))
most_common_t0 = word_freq_t0.most_common()

In [ ]:
most_common_t1

In [ ]:
most_common_t0

Baseline model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train['text_modif'], train['target'], test_size=0.3, stratify = train['target'],
    random_state=42)

In [ ]:
vectorizer = CountVectorizer(min_df = 0.0008)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)
bow_df_train = pd.DataFrame(X_train_bow.toarray())
bow_df_test = pd.DataFrame(X_test_bow.toarray())
bow_df_train.columns = vectorizer.get_feature_names()
bow_df_test.columns = vectorizer.get_feature_names()
print(bow_df_train.columns)
print(bow_df_train.shape)

No when we have all the architecture done, let's see the results of our baseline model.

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)
predictions = clf.predict(X_test_bow)
print(f1_score(y_test, predictions, average='micro'))

Feature engeneering part.

In [ ]:
from wordcloud import STOPWORDS
stop_words = ["https", "amp", "CO",] + list(STOPWORDS)
cloud_comment_words = WordCloud(stopwords  = stop_words,
    background_color="black").generate(' '.join(list(train['text_modif'])))
plt.imshow(cloud_comment_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_comment_words_true = WordCloud(stopwords  = stop_words,
    background_color="black").generate(' '.join(list(train['text_modif'][train['target'] == 0])))
plt.imshow(cloud_comment_words_true, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_comment_words_false = WordCloud(stopwords  = stop_words,
    background_color="black").generate(' '.join(list(train['text_modif'][train['target'] == 1])))
plt.imshow(cloud_comment_words_false, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
train['commenet_polar'] = train['text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['commenet_polar'], num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Count')
plt.title('Histogram of Total polarity')
plt.show()

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['commenet_polar'][train['target'] == 0], num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Count')
plt.title('Histogram of polarity in True comments')
plt.show()

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['commenet_polar'][train['target'] == 1], num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Count')
plt.title('Histogram of polarity in False comments')
plt.show()

In [ ]:
train['commenet_subject'] = train['text'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['commenet_subject'], num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Count')
plt.title('Histogram of Total subjectivity')
plt.show()

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['commenet_subject'][train['target'] == 0], num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('subjectivity')
plt.ylabel('Count')
plt.title('Histogram of subjectivity in True comments')
plt.show()

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['commenet_subject'][train['target'] == 1], num_bins, facecolor='blue', alpha=0.5)
plt.xlabel('subjectivity')
plt.ylabel('Count')
plt.title('Histogram of subjectivity in False comments')
plt.show()

In [ ]:
cloud_key_words = WordCloud(
    background_color="black").generate(' '.join(list(train['keyword'][train['keyword'].notna()])))
plt.imshow(cloud_key_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
train['keyword'][train['keyword'].notna() & train['keyword'].str.contains('20')].unique()

In [ ]:
train['keyword'] = np.where(train['keyword'].notna(), train['keyword'].str.replace('%20', ' and '), train['keyword'])

In [ ]:
print(train['keyword'][train['target'] == 1].value_counts(dropna = False, normalize = True))

In [ ]:
cloud_key_words = WordCloud(
    background_color="black").generate(' '.join(list(train['keyword'][(train['keyword'].notna()) &
                                                                      (train['target'] == 0)]) ) )
plt.imshow(cloud_key_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_key_words = WordCloud(
    background_color="black").generate(' '.join(list(train['keyword'][(train['keyword'].notna()) &
                                                                      (train['target'] == 1)]) ) )
plt.imshow(cloud_key_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

Let's perform a sentiment analysis on the key words extracting their polarity and subjectivity.

In [ ]:
train['key_word_polar'] = np.where(
    train['keyword'].notna(), train['keyword'].apply(lambda x: TextBlob(str(x)).sentiment.polarity), train['keyword'])

In [ ]:
train['key_word_subj'] = np.where(
    train['keyword'].notna(), train['keyword'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity), train['keyword'])

In [ ]:
train['key_word_polar'] = np.where(train['key_word_polar'].notna(),
                                   train['key_word_polar'].apply(lambda x: round(x*10)), train['key_word_polar'])

In [ ]:
train['key_word_polar'][train.target == 0].describe()

In [ ]:
train.keyword[train.target == 1].value_counts(dropna = False,normalize = True)

In [ ]:
train['comment_length'] = train['text'].apply(len)

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['comment_length'], num_bins, facecolor='red', alpha=0.5)
plt.xlabel('comment_length')
plt.ylabel('Count')
plt.title('Histogram of Total comment_length')
plt.show()

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['comment_length'][train['target'] == 0], num_bins, facecolor='green', alpha=0.5)
plt.xlabel('comment_length')
plt.ylabel('Count')
plt.title('Histogram of comment_length in True comments')
plt.show()

In [ ]:
num_bins = 25
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(train['comment_length'][train['target'] == 1], num_bins, facecolor='green', alpha=0.5)
plt.xlabel('comment_length')
plt.ylabel('Count')
plt.title('Histogram of comment_length in False comments')
plt.show()

Next, we will compute the mean number of different parts of speech and name enteties used in fake and real comments and compare the values. If there is a remarkable difference, then there is a good chance that using the created feature in fake news detectors will improve its performance.

In [ ]:
def spacy_counter(df, type_, text_column, target_column):
    '''Functions that return a number part of speech or entetis in a string.
    '''
    def partOfSpeachRec(text,tag):
        doc = nlp(text)
        pos = [token.pos_ for token in doc]
        return pos.count(tag)
    def entetiesRecog(text, tag):
        doc = nlp(text)
        labels = [ent.label_ for ent in doc.ents]
        return labels.count(tag)
    
    if type_ == 'part_of_speech':
        aux_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON',
                       'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']
        function = partOfSpeachRec
    elif type_ == 'entety_label':
        aux_list = ['PERSON', 'ORG', 'GPE', 'NORP', 'FAC', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART','LAW', 'LANGUAGE', 
                    'DATE','TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']
        function = entetiesRecog
    for i in aux_list:
        df[i] = df[text_column].apply(lambda x: function(x, i))
        real_mean = df[df[target_column] == 0][i].mean()
        fake_mean = df[df[target_column] == 1][i].mean()
        mean_diff = abs(real_mean - fake_mean)
        
        real_median = df[df[target_column] == 0][i].median()
        fake_median = df[df[target_column] == 1][i].median()
        
        print(i)
        print("Mean no. of %s in real and fake comments are %.2f and %.2f respectively with the difference of %.2f"%(
            i, real_mean, fake_mean, mean_diff))
        print("Median no. of %s in real and fake comments are %.2f and %.2f respectively"%(i, real_median, fake_median))

    return df

In [ ]:
train['text'] = train['text'].apply(lambda x: x.replace('#', '').replace('@', ''))

In [ ]:
spacy_counter(train, 'part_of_speech', 'text', 'target')

1. It looks like the number of nouns (NOUN with the mean difference equal to 0.74) and proper names (PROPN with the mean difference equal to 0.70) might do do some good to our model. Let's take a look at the number of parts of speech in the comments.

In [ ]:
spacy_counter(train, 'entety_label', 'text', 'target')

The most differences can be detected in 'Countries, Cities and States' (GPE, 0.25) and 'Numerals' ( CARDINAL, 0.11) labels, so we are going to include them too.

In [ ]:
def part_speech(string, tag):
    doc = nlp(string)
    token_list = [token.pos_ for token in doc]
    return token_list.count(tag)

In [ ]:
train['NOUN'] = train['text'].apply(lambda x: part_speech(x,'NOUN'))
train['PROPN'] = train['text'].apply(lambda x: part_speech(x,'PROPN'))

In [ ]:
def ent(string, tag):
    doc = nlp(string)
    ent_list = [ent.label_ for ent in doc.ents]
    return ent_list.count(tag)

In [ ]:
train['GPE'] = train['text'].apply(lambda x: ent(x,'GPE'))
train['CARDINAL'] = train['text'].apply(lambda x: ent(x,'CARDINAL'))

Let's compare the results of our base model using newly made variables.

In [ ]:
X_train_plus, X_test_plus, y_train_plus, y_test_plus = train_test_split(
    train[['NOUN', 'PROPN', 'GPE', 'CARDINAL']], train['target'], test_size=0.3, stratify = train['target'],
    random_state=42)

In [ ]:
X_train_plus.reset_index(drop=True, inplace=True)
X_test_plus.reset_index(drop=True, inplace=True)
y_train_plus.reset_index(drop=True, inplace=True)
y_test_plus.reset_index(drop=True, inplace=True)

In [ ]:
X_train_plus.head()

In [ ]:
X_train_plus_ad = pd.concat([X_train_plus, bow_df_train], axis = 1)
X_test_plus_ad = pd.concat([X_test_plus, bow_df_test], axis = 1)

In [ ]:
clf.fit(X_train_plus_ad, y_train_plus)
predictions_plus = clf.predict(X_test_plus_ad)
print('Base model with additional variables:')
print(round(f1_score(y_test_plus, predictions_plus, average='micro'), 2))
print('Base model with no additional variables:')
print(round(f1_score(y_test, predictions, average='micro'), 2))

We gain a bit in F1 score, so let's leave the additional variables.

Let's try playnig with N-grams models.

In [ ]:
# Generating 1 ngrams
vect_1 = CountVectorizer(ngram_range=(1,1))
train_X_n1 = vect_1.fit_transform(X_train)
test_X_n1 = vect_1.transform(X_test)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X_n1, y_train)
pr_n1 = clf.predict(test_X_n1)

In [ ]:
# Generating 2 ngrams
vect_2 = CountVectorizer(ngram_range=(1,2))
train_X_n2 = vect_2.fit_transform(X_train)
test_X_n2 = vect_2.transform(X_test)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X_n2, y_train)
pr_n2 = clf.predict(test_X_n2)

In [ ]:
# Generating 5 ngrams
vect_3 = CountVectorizer(ngram_range=(1,5))
train_X_n3 = vect_3.fit_transform(X_train)
test_X_n3 = vect_3.transform(X_test)

# Fit classifier
clf = MultinomialNB()
clf.fit(train_X_n3, y_train)
pr_n3 = clf.predict(test_X_n3)

In [ ]:
print('N-1 model with no additional variables:')
print(f1_score(y_test, pr_n1, average='micro'))

print('N-2 model with no additional variables:')
print(f1_score(y_test, pr_n2, average='micro'))

print('N-5 model with no additional variables:')
print(f1_score(y_test, pr_n3, average='micro'))

In [ ]:
bow_df_train_n = pd.DataFrame(train_X_n3.toarray())
bow_df_test_n = pd.DataFrame(test_X_n3.toarray())

In [ ]:
X_train_plus_n = pd.concat([X_train_plus, bow_df_train_n], axis = 1)
X_test_plus_n = pd.concat([X_test_plus, bow_df_test_n], axis = 1)

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_plus_n, y_train_plus)
pr_n3_add = clf.predict(X_test_plus_n)

In [ ]:
print('N-5 model with the additional variables:')
print(round(f1_score(y_test, pr_n3_add, average='micro'), 2))

In [ ]:
tfdf = TfidfVectorizer(min_df = 0.0001)
X = tfdf.fit_transform(train['text_modif'])
Y = train['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify = Y, random_state=42)

In [ ]:
clf.fit(X_train, y_train)
tfdf_pred = clf.predict(X_test)

print('TFDF model with the additional variables:')
print(f1_score(y_test, tfdf_pred, average='micro'))